Things to try to increase performance: 
-even more data
-separate models for different genres
-topic model
-ridge regression, lasso regression
-bi grams, n grams
-sentiment analysis
-Look for transformations opps, etc
-model only using genre

In [183]:
import pandas as pd
from textblob import TextBlob
from textblob import Word
from gensim import corpora, models, similarities
import os
import tempfile
import gensim
from sklearn.linear_model import LinearRegression
import math
import numpy as np
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
import tensorflow as tf

TEMP_FOLDER = tempfile.gettempdir()

In [184]:
links = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/links2.csv', header = None)
text = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/reviewtext.csv', header = None)

In [185]:
links2 = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/links3.csv', header = None)
text2 = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/reviewtext2.csv', header = None)

In [186]:
#prep 2nd set of data, which has lots of links that didn't work

links2[5] = text2
links2.drop_duplicates([5], inplace = True)
links2 = links2[links2[5] != '[]']
links2 = links2[links2[5] != '[]']
links2.dropna(subset = [5],inplace = True)

In [187]:
links[5] = text

In [188]:
links.drop_duplicates([5], inplace = True)


In [189]:
links = pd.concat([links,links2], axis = 0)
links = links.reset_index()
links = links.drop('index', axis = 1)

In [190]:
links.shape

(2903, 6)

In [ ]:
#adding average genre incrementor to score
genregroups = links.groupby([3]).mean()
totalmean = links[1].mean()
genregroups.reset_index(inplace = True)
genregroups[2] = totalmean
genregroups['adj'] = genregroups[1]-genregroups[2]
links =  links.merge(genregroups[['adj', 3]], on=[3])
links

In [192]:
text = links[5]
text.shape

(2893,)

In [193]:
#lowercase text
text = text.apply(lambda x: " ".join(x.lower() for x in x.split()))
text.head()

0    ["with all of the development muscle behind ea...
1    [' ', 'there aren’t many games that make me fe...
2    ["all too often, the single-player campaign of...
3    [' ', 'call of duty: black ops 3 flung the ser...
4    ['developer treyarch touted that, for call of ...
Name: 5, dtype: object

In [194]:
#remove brackets
text = text.apply(lambda x: x[1:-1])
text.head()

0    "with all of the development muscle behind ea ...
1    ' ', 'there aren’t many games that make me fee...
2    "all too often, the single-player campaign of ...
3    ' ', 'call of duty: black ops 3 flung the seri...
4    'developer treyarch touted that, for call of d...
Name: 5, dtype: object

In [195]:
#remove new line
text = text.replace(r'\\n','', regex=True)
text.head()

0    "with all of the development muscle behind ea ...
1    ' ', 'there aren’t many games that make me fee...
2    "all too often, the single-player campaign of ...
3    ' ', 'call of duty: black ops 3 flung the seri...
4    'developer treyarch touted that, for call of d...
Name: 5, dtype: object

In [196]:
#remove punc
text = text.str.replace('[^\w\s]','')
text.head()

0    with all of the development muscle behind ea d...
1      there arent many games that make me feel as ...
2    all too often the singleplayer campaign of a p...
3      call of duty black ops 3 flung the series in...
4    developer treyarch touted that for call of dut...
Name: 5, dtype: object

In [197]:
#remove stop words
from nltk.corpus import stopwords
stop = stopwords.words('english')
text = text.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
text.head()

0    development muscle behind ea dices massive sho...
1    arent many games make feel cool warframe im bu...
2    often singleplayer campaign primarily multipla...
3    call duty black ops 3 flung series thrustjumpi...
4    developer treyarch touted call duty black ops ...
Name: 5, dtype: object

In [198]:
freq = pd.Series(' '.join(text[0]).split()).value_counts()[-5:]
#remove uncommon words
freq = list(freq.index)
text = text.apply(lambda x: " ".join(x for x in x.split() if x not in freq))
text.head()

0    development muscle behind ea dices massive sho...
1    arent many games make feel cool warframe im bu...
2    often singleplayer campaign primarily multipla...
3    call duty black ops flung series thrustjumping...
4    developer treyarch touted call duty black ops ...
Name: 5, dtype: object

In [199]:
###remove common words###
freq = pd.Series(' '.join(text[0]).split()).value_counts()[:10]
freq = list(freq.index)
text = text.apply(lambda x: " ".join(x for x in x.split() if x not in freq))
#Visualize
text.head()

0    development muscle behind ea dices massive sho...
1    arent many games make feel cool warframe im bu...
2    often singleplayer campaign primarily multipla...
3    call duty black ops flung series thrustjumping...
4    developer treyarch touted call duty black ops ...
Name: 5, dtype: object

In [200]:
###Spelling correction###

text[:5].apply(lambda x: str(TextBlob(x).correct()))

###Tokenization (dividing the text into a sequence of words or sentences)###
TextBlob(text[0][1]).words

#lemmatize
text = text.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
text.head()

0    development muscle behind ea dice massive shoo...
1    arent many game make feel cool warframe im bul...
2    often singleplayer campaign primarily multipla...
3    call duty black ops flung series thrustjumping...
4    developer treyarch touted call duty black ops ...
Name: 5, dtype: object

In [201]:
text = text.to_frame()
text['score'] = links[1]
text['adj'] = links['adj']

In [259]:
#train val split

trainval = text.sample(800)
train = text.loc[~text.index.isin(trainval.index), :]
y = train['score']
valy = trainval[['score', 'adj']].copy()
trainval.drop(['score','adj'], axis = 1, inplace = True)
train.drop(['score', 'adj'], axis = 1, inplace = True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [260]:
#for train values
textlist = train.values.tolist()

In [261]:
textlist = [str(word) for word in textlist]

In [262]:
texts = [[word for word in document.lower().split() ]
         for document in textlist]

In [263]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=5, no_above=0.6, keep_n=35000)

In [264]:
dictionary.save(os.path.join(TEMP_FOLDER, 'igntfidf.dict'))

In [265]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'igntfidf.mm'), corpus)  # store to disk, for later use

In [266]:
tfidf = models.TfidfModel(corpus)

In [267]:
corpus_tfidf = tfidf[corpus]

In [268]:
all_topics_csr = gensim.matutils.corpus2csc(corpus_tfidf)

In [269]:
all_topics_csr.shape

(13288, 2093)

In [270]:
all_topics_numpy = all_topics_csr.T.toarray()

In [271]:
all_topics_numpy

array([[0.04033982, 0.01397278, 0.04444697, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [272]:
#validation array
vallist = trainval.values.tolist()
vallist = [str(word) for word in vallist]
valtexts = [[word for word in document.lower().split() ]
         for document in vallist]
valcorpus = [dictionary.doc2bow(valtext) for valtext in valtexts]
testcorpus_tfidf = tfidf[valcorpus]
all_topics_csr2 = gensim.matutils.corpus2csc(testcorpus_tfidf)
all_topics_numpytest = all_topics_csr2.T.toarray()

In [273]:
all_topics_numpytest

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.03026141, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [274]:
print(all_topics_numpytest.shape)
print(all_topics_numpy)

(800, 13288)
[[0.04033982 0.01397278 0.04444697 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


## XGBoost

In [ ]:
#bayesian opt
yarray = y.values
dtrain = xgb.DMatrix(all_topics_numpy, label=yarray)
def xgb_evaluate(max_depth, gamma, colsample_bytree):
    params = {'eval_metric': 'rmse',
              'max_depth': int(max_depth),
              'subsample': 0.8,
              'eta': 0.1,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree}
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain, num_boost_round=100, nfold=3)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 7),
                                            'gamma': (0, 1),
                                            'colsample_bytree': (0.3, 0.9)})

xgb_bo.maximize(init_points=3, n_iter=5, acq='ei')



In [357]:
param = xgb_bo.max
param

{'target': -1.1006746666666667,
 'params': {'colsample_bytree': 0.8976729774092942,
  'gamma': 0.9971962663965198,
  'max_depth': 4.321612175930944}}

In [340]:
yarray = y.values
xgbreg = xgb.XGBRegressor()
xgbreg.fit(all_topics_numpy,yarray)

predictions2 = xgbreg.predict(all_topics_numpytest)
predictionsdf2 = pd.DataFrame(predictions2)
valy = valy.reset_index()
valy.drop(['index'], axis = 1, inplace = True)
predictionsdf2['true'] = valy
predictionsdf2['diff'] = predictionsdf2['true'] - predictionsdf2[0]
predictionsdf2['diff'] = predictionsdf2['true'] - predictionsdf2[0]
predictionsdf2['diff'] = predictionsdf2['diff']**2
diff = predictionsdf2['diff'].values
diff = np.mean(diff)
diff = math.sqrt(diff)
diff

1.0728785941278771

## Lasso Regression

In [338]:
#find best alpha for lasso regression

alphas = [1e-15, 1e-10, 1e-8, 1e-5,1e-4, 1e-3,1e-2, 1, 5, 10]
alphas2 = [1e-5,1.5e-5,1e-4, 1.5e-4,1e-3]
def runlasso(alpha):
    global valy
    yarray = y.values
    lasso = Lasso(alpha = alpha, max_iter=1e5)
    lasso.fit(all_topics_numpy,yarray)

    predictions3 = lasso.predict(all_topics_numpytest)
    predictionsdf3 = pd.DataFrame(predictions3)
    valy = valy.reset_index()
    valy.drop(['index'], axis = 1, inplace = True)
    predictionsdf3['true'] = valy
    predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
    predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
    predictionsdf3['diff'] = predictionsdf3['diff']**2
    diff = predictionsdf3['diff'].values
    diff = np.mean(diff)
    diff = math.sqrt(diff)
    return diff
    
test = [runlasso(x) for x in alphas2]
print(test)
    
    
    

[1.1252147868577373, 1.1126873118616674, 1.0177467770085127, 0.9907478349117224, 1.1576718275231785]


In [154]:
yarray.shape

(2103,)

In [339]:
yarray = y.values
lasso = Lasso(alpha = 1.5e-4, max_iter=1e5)
lasso.fit(all_topics_numpy,yarray)
predictions3 = lasso.predict(all_topics_numpytest)
predictionsdf3 = pd.DataFrame(predictions3)
valy = valy.reset_index()
valy.drop(['index'], axis = 1, inplace = True)
predictionsdf3['true'] = valy
predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
predictionsdf3['diff'] = predictionsdf3['diff']**2
diff = predictionsdf3['diff'].values
diff = np.mean(diff)
diff = math.sqrt(diff)
diff


0.9907478349117224

## OLS

In [275]:
yarray = y.values
lm = LinearRegression()
lm.fit(all_topics_numpy,yarray)
predictions = lm.predict(all_topics_numpytest)
predictionsdf = pd.DataFrame(predictions)

In [276]:
predictionsdf.describe()

,0
count,800.000000
mean,7.300570
std,1.213731
min,3.050119
25%,6.556257
50%,7.349701
75%,8.156121
max,10.332025


In [277]:
valy = valy.reset_index()
valy

,index,score,adj
0,1300,4.5,0.302714
1,2852,8.0,0.140039
2,2255,9.0,0.002149
3,600,7.9,-0.349524
4,1510,5.5,0.289281
5,2308,8.5,0.002149
6,1228,9.5,0.218636
7,2877,7.5,0.173372
8,1823,9.0,0.033249
9,2067,8.5,0.317635


In [278]:
valy.drop(['index'], axis = 1, inplace = True)
valy

,score,adj
0,4.5,0.302714
1,8.0,0.140039
2,9.0,0.002149
3,7.9,-0.349524
4,5.5,0.289281
5,8.5,0.002149
6,9.5,0.218636
7,7.5,0.173372
8,9.0,0.033249
9,8.5,0.317635


In [279]:
predictionsdf2 = predictionsdf
predictionsdf['true'] = valy['score']
predictionsdf2['true'] = valy['score']
predictionsdf2['adj'] = valy['adj']
predictionsdf2

,0,true,adj
0,5.465251,4.5,0.302714
1,7.186878,8.0,0.140039
2,8.496204,9.0,0.002149
3,7.396167,7.9,-0.349524
4,6.364052,5.5,0.289281
5,7.840346,8.5,0.002149
6,9.807379,9.5,0.218636
7,6.745549,7.5,0.173372
8,7.917756,9.0,0.033249
9,7.891965,8.5,0.317635


In [280]:
predictionsdf['diff'] = predictionsdf['true'] - predictionsdf[0]
predictionsdf2['diff'] = predictionsdf2['true'] - predictionsdf2[0] + predictionsdf['adj']
predictionsdf2

,0,true,adj,diff
0,5.465251,4.5,0.302714,-0.662537
1,7.186878,8.0,0.140039,0.953161
2,8.496204,9.0,0.002149,0.505945
3,7.396167,7.9,-0.349524,0.154309
4,6.364052,5.5,0.289281,-0.574770
5,7.840346,8.5,0.002149,0.661804
6,9.807379,9.5,0.218636,-0.088744
7,6.745549,7.5,0.173372,0.927824
8,7.917756,9.0,0.033249,1.115493
9,7.891965,8.5,0.317635,0.925669


In [333]:
#let's also plot histogram of scores




In [334]:
#residual plot here




In [281]:
#RMSE
predictionsdf['diff'] = predictionsdf['true'] - predictionsdf[0]
predictionsdf['diff'] = predictionsdf['diff']**2
diff = predictionsdf['diff'].values
diff = np.mean(diff)
diff = math.sqrt(diff)
diff

#diff = predictionsdf['diff'].values.tolist()
#diff = math.sqrt(np.mean(diff**2))
#diff

1.0412305102697863

In [283]:
#RMSE
predictionsdf2['diff'] = predictionsdf2['true'] - predictionsdf2[0] + predictionsdf2['adj']
predictionsdf2['diff'] = predictionsdf2['diff']**2
diff = predictionsdf2['diff'].values
diff = np.mean(diff)
diff = math.sqrt(diff)
diff

1.1611732907513854

In [228]:
arravg = np.mean(yarray)
predictionsdf['avg'] = arravg
predictionsdf['avgdiff'] = predictionsdf['avg'] - predictionsdf['true']
predictionsdf['avgdiff'] = predictionsdf['avgdiff']**2
avgdiff = predictionsdf['avgdiff'].values
avgdiff = np.mean(avgdiff)
avgdiff = math.sqrt(avgdiff)
avgdiff

1.6069504481156325